In [20]:
import os
import json
import requests
from langchain.retrievers import WikipediaRetriever
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage

from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from typing import Type

llm = ChatOpenAI(temperature=0.1, model_name = "gpt-3.5-turbo-1106")
alpha_vantage_api_key = os.environ.get("GI7UXI89QUWWJFK5")

class WikiArgsSchema(BaseModel):
    term: str = Field(description="The search of the company.Example: Apple company")
    
class WikiSearchTool(BaseTool):
    name = "WikiSearchTool"
    description = """
    Use this tool to find the company's information.
    Example term: Apple company
    """
    args_schema: Type[WikiArgsSchema] = WikiArgsSchema
    def _run(self, term):
        retriever = WikipediaRetriever(top_k_results=4, lang="ko")
        docs = retriever.get_relevant_documents(term)
        return docs

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for.Example query: Stock Market Symbol for Apple Company")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema
    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

#

class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(description="Stock symbol of the company.Example: AAPL,TSLA")

class CompanyOverviewTool(BaseTool):
    name="CompanyOverview"
    description="""
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyIncomeStatementTool(BaseTool):
    name="CompanyIncomeStatement"
    description="""
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]
    
class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockperformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        response = r.json()
        return list(response["Weekly Time Series"].items())[:200]


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        # 주식 심볼을 아는 데 사용
        CompanyOverviewTool(),
        # 정보를 얻는데 사용
        CompanyIncomeStatementTool(),
        # llm이 사용 안 할 수도 있다. llm이 더 많은 정보가 필요하다고 생각할 때만 사용
        CompanyStockPerformanceTool(),
        WikiSearchTool(),
    ],
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
        You are a hedge fund manager.
            
        You evaluate a company and provide your opinion and reasons why the stock is a buy or not.
            
        Consider the performance of a stock, the company overview and the income statement.
            
        Be assertive in your judgement and recommend the stock or advise the user against it.                                
        """
        )
    },
)

# streamlit

st.set_page_config(
    page_title="InvestorGPT",
    page_icon="💼",
)

st.markdown(
    """
    # InvestorGPT
            
    Welcome to InvestorGPT.
            
    Write down the name of a company and our Agent will do the research for you.
"""
)

company = "Research about the XZ backdoo"
# company = "cloudflare"

result = agent.invoke(company)

file = open(f"{company}.txt", "w", encoding="CP949")
file.writelines(result["output"])
file.close()




> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'XZ backdoor'}`


The XZ Backdoor: Everything You Need to Know. Details are starting to emerge about a stunning supply chain attack that sent the open source software community reeling. On Friday, a lone Microsoft ... On March 28, 2024 a backdoor was identified in XZ Utils. This vulnerability, CVE-2024-3094 with a CVSS score of 10 is a result of a software supply chain compromise impacting versions 5.6.0 and 5.6.1 of XZ Utils. The U.S. Cybersecurity and Infrastructure Security Agency (CISA) has recommended organizations to downgrade to a previous non-compromised XZ Utils version. The version of XZ should be carefully checked, as versions 5.6.0 and 5.6.1 contain the backdoor. It is advised to revert to a previous known safe version of XZ Utils, such as 5.4. Software supply ... CVE-2024-3094 is a vulnerability discovered in the open-source library XZ Utils that stems from malicious code that 